# Transmon drive hamiltonian

In [1]:
from IPython.display import display
from sympy import Function
from sympy import I as i
from sympy import Matrix, Symbol, exp, sqrt
from sympy.physics.quantum import Dagger as dag

t = Symbol("t", real=True)
alpha = Symbol("alpha", real=True)
omega = Symbol("omega", real=True)

In [2]:
E = Function("E", real=True)(t)
E_x = Function("E_x", real=True)(t)
E_y = Function("E_y", real=True)(t)
Omega = Function("Omega", real=False)(t)
gamma = Function("gamma", real=False)(t)

## Operators

### Ladder operators

$$
\begin{aligned}
b &= \begin{bmatrix} 0 & 1 & 0 \\ 0 & 0 & \sqrt{2} \\ 0 & 0 & 0 \end{bmatrix}
\\
b^\dagger &= \begin{bmatrix} 0 & 0 & 0 \\ 1 & 0 & 0 \\ 0 & \sqrt{2} & 0 \end{bmatrix}
\\
N &= \begin{bmatrix} 0 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 2 \end{bmatrix}
\end{aligned}
$$

In [3]:
b = Matrix(
    [
        [0, 1, 0],
        [0, 0, sqrt(2)],
        [0, 0, 0],
    ]
)

bd = dag(b)

N = bd * b

display(b, bd, N)

Matrix([
[0, 1,       0],
[0, 0, sqrt(2)],
[0, 0,       0]])

Matrix([
[0,       0, 0],
[1,       0, 0],
[0, sqrt(2), 0]])

Matrix([
[0, 0, 0],
[0, 1, 0],
[0, 0, 2]])

### System hamiltonian

$$
\begin{aligned}
H_1 &= \omega b^\dagger b = \begin{bmatrix} 0 & 0 & 0 \\ 0 & \omega & 0 \\ 0 & 0 & 2\omega \end{bmatrix}
\\
H_2 &= \frac{\alpha}{2} b^{\dagger 2} b^2 = \begin{bmatrix} 0 & 0 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & \alpha \end{bmatrix}
\\
H_\mathrm{sys} &= H_1 + H_2 = \begin{bmatrix} 0 & 0 & 0 \\ 0 & \omega & 0 \\ 0 & 0 & 2\omega + \alpha \end{bmatrix}
\end{aligned}
$$

In [4]:
H_1 = omega * bd * b
H_1

Matrix([
[0,     0,       0],
[0, omega,       0],
[0,     0, 2*omega]])

In [5]:
H_2 = alpha / 2 * bd * bd * b * b
H_2

Matrix([
[0, 0,     0],
[0, 0,     0],
[0, 0, alpha]])

In [6]:
H_sys = H_1 + H_2
H_sys

Matrix([
[0,     0,               0],
[0, omega,               0],
[0,     0, alpha + 2*omega]])

### Drive hamiltonian

#### Real expression

$$
\begin{aligned}
V &= E (b + b^\dagger) = \begin{bmatrix} 0 & E & 0 \\ E & 0 & \sqrt{2} E \\ 0 & \sqrt{2} E & 0 \end{bmatrix}
\end{aligned}
$$

$$
\begin{aligned}
E &= E_x \cos\omega t + E_y \sin\omega t
\end{aligned}
$$


#### Complex expression

$$
\begin{aligned}
V &= \frac{1}{2} (\gamma + \gamma^*) (b + b^\dagger) = \frac{1}{2} \begin{bmatrix} 0 & \gamma + \gamma^* & 0 \\ \gamma + \gamma^* & 0 & \sqrt{2}(\gamma + \gamma^*) \\ 0 & \sqrt{2}(\gamma + \gamma^*) & 0 \end{bmatrix}
\end{aligned}
$$

$$
\begin{aligned}
\gamma &= \Omega e^{-i\omega t}
\\
&= (E_x + i E_y) e^{-i\omega t}
\end{aligned}
$$


#### Equivalence

$$
\begin{aligned}
\gamma + \gamma^* &= (E_x + i E_y) e^{-i\omega t} + (E_x - i E_y) e^{i\omega t}
\\
&= E_x (e^{-i\omega t} + e^{i\omega t}) + i E_y (e^{-i\omega t} - e^{i\omega t})
\\
&= 2 E_x \cos\omega t + 2 E_y \sin\omega t
\\
&= 2 E
\end{aligned}
$$

In [7]:
V = (gamma + dag(gamma)) / 2 * (b + bd)
V

Matrix([
[                              0,           gamma(t)/2 + Dagger(gamma(t))/2,                                         0],
[gamma(t)/2 + Dagger(gamma(t))/2,                                         0, sqrt(2)*(gamma(t)/2 + Dagger(gamma(t))/2)],
[                              0, sqrt(2)*(gamma(t)/2 + Dagger(gamma(t))/2),                                         0]])

In [8]:
H = H_sys + V
H

Matrix([
[                              0,           gamma(t)/2 + Dagger(gamma(t))/2,                                         0],
[gamma(t)/2 + Dagger(gamma(t))/2,                                     omega, sqrt(2)*(gamma(t)/2 + Dagger(gamma(t))/2)],
[                              0, sqrt(2)*(gamma(t)/2 + Dagger(gamma(t))/2),                           alpha + 2*omega]])

In [9]:
U_0 = exp(-i * H_1 * t)
U_0

Matrix([
[1,               0,                 0],
[0, exp(-I*omega*t),                 0],
[0,               0, exp(-2*I*omega*t)]])

In [10]:
H_rot = dag(U_0) * H * U_0 - i * dag(U_0) * U_0.diff(t)
H_rot

Matrix([
[                                               0,        (gamma(t)/2 + Dagger(gamma(t))/2)*exp(-I*omega*t),                                                         0],
[(gamma(t)/2 + Dagger(gamma(t))/2)*exp(I*omega*t),                                                        0, sqrt(2)*(gamma(t)/2 + Dagger(gamma(t))/2)*exp(-I*omega*t)],
[                                               0, sqrt(2)*(gamma(t)/2 + Dagger(gamma(t))/2)*exp(I*omega*t),                                                     alpha]])

In [11]:
H_rot.subs({gamma: Omega * exp(-i * omega * t)}).expand()

Matrix([
[                                               0,                Omega(t)*exp(-2*I*omega*t)/2 + Dagger(Omega(t))/2,                                                                 0],
[Omega(t)/2 + exp(2*I*omega*t)*Dagger(Omega(t))/2,                                                                0, sqrt(2)*Omega(t)*exp(-2*I*omega*t)/2 + sqrt(2)*Dagger(Omega(t))/2],
[                                               0, sqrt(2)*Omega(t)/2 + sqrt(2)*exp(2*I*omega*t)*Dagger(Omega(t))/2,                                                             alpha]])

#### RWA: Rotating Wave Approximation

$$
\begin{aligned}
V_\mathrm{rwa} &= \frac{1}{2} (\gamma b^\dagger + \gamma^* b) = \begin{bmatrix} 0 & \gamma^* & 0 \\ \gamma & 0 & \sqrt{2}\gamma^* \\ 0 & \sqrt{2}\gamma & 0 \end{bmatrix}
\\
\gamma &= \Omega e^{-i\omega t}
\\
&= (E_x + i E_y) e^{-i\omega t}
\end{aligned}
$$

In [12]:
V_rwa = (gamma * bd + dag(gamma) * b) / 2
V_rwa

Matrix([
[         0, Dagger(gamma(t))/2,                          0],
[gamma(t)/2,                  0, sqrt(2)*Dagger(gamma(t))/2],
[         0, sqrt(2)*gamma(t)/2,                          0]])

In [13]:
H_rwa = H_sys + V_rwa
H_rwa

Matrix([
[         0, Dagger(gamma(t))/2,                          0],
[gamma(t)/2,              omega, sqrt(2)*Dagger(gamma(t))/2],
[         0, sqrt(2)*gamma(t)/2,            alpha + 2*omega]])

In [14]:
H_rwa_rot = dag(U_0) * H_rwa * U_0 - i * dag(U_0) * U_0.diff(t)
H_rwa_rot

Matrix([
[                        0, exp(-I*omega*t)*Dagger(gamma(t))/2,                                          0],
[gamma(t)*exp(I*omega*t)/2,                                  0, sqrt(2)*exp(-I*omega*t)*Dagger(gamma(t))/2],
[                        0,  sqrt(2)*gamma(t)*exp(I*omega*t)/2,                                      alpha]])

In [15]:
H_rwa_rot.subs({gamma: Omega * exp(-i * omega * t)})

Matrix([
[         0, Dagger(Omega(t))/2,                          0],
[Omega(t)/2,                  0, sqrt(2)*Dagger(Omega(t))/2],
[         0, sqrt(2)*Omega(t)/2,                      alpha]])